In [1]:
import pandas as pd
from io import open
import json as json
import re
import numpy as np

In [2]:
archivo = open("../osm/edificio_este.osm","r")

osm = archivo.readlines()

In [42]:


def get_object(osm,OSM_OBJETO):
    # identifica donde inicia un objeto y donde 
    # termina, y los pone en una lista    
    objeto = []
    coma   = []
    for i,renglon in enumerate(osm):
        if renglon.startswith("OS:"): #localiza el inicio de un objeto
            objeto.append(i)
        if ";" in renglon:  #localiza el final de un objeto
            coma.append(i)
#     en os_lista, guarda donde inicia y termina cada objeto
    osm_lista = []
    for i,j in zip(objeto,coma):
        osm_lista.append(osm[i:j])
# separa en dos listas, lista objeto y resto del osm
    objeto_lista = []
    resto_lista  = []
    for objeto in osm_lista:
        if OSM_OBJETO in objeto:
            objeto_lista.append(objeto)
        else:
            resto_lista.append(objeto)
#             Limpia la lista que contiene el objeto
    lista_objetos = []
    for objeto in objeto_lista:
        lista_objeto = []
        objeto[0]= objeto[0].replace(",","")
        for campo in objeto:
            lista_objeto.append(campo.replace("\n","").
                                 replace("","").
                                 replace("{","").
                                 replace("}","").
                                 replace("!- ","").
                                 strip())
        lista_objetos.append(lista_objeto)
    
    diccionario = {}
    for objeto in lista_objetos:
        nombre_objeto,_ = objeto[2].split(",")
        tmp = {}
        for propiedad in range(1,len(objeto)):
            try:
                valor,nombre = objeto[propiedad].split(",")
                try:
                    valor = float(valor)
                except:
                    pass

            except:
                pass
            tmp.update({nombre:valor})
        del tmp["Name"]
        diccionario.update({nombre_objeto:tmp})
# diccionario
    return lista_objetos,diccionario
materiales, diccionario = load_object(osm,"OS:Material,\n")
for objeto in diccionario:
    print(objeto)
diccionario['madera_L2p2_a0pX']["Thermal Absorptance"]

concreto_fc150_granzon_L8_a0pX
concreto_fc150_L6_a0pX
concreto_fc250_L30_a0pX
madera_L2p2_a0pX
marmol_L2_a0pX
concreto_fc250_L20_a0pX
mortero1:4_L1_a0pX
loseta_Lx_a0pX
impermeabilizante_LX_a0p3
fachaleta_L1_a0p3
concreto_fc200_L15_a0pX
durock_L1p27_a0p3
concreto_fc250_L15_a0pX
concreto_fc150_marmol_L8_a0pX
concreto_fc150_L8_a0pX
crest_L1_a0pX
tabique_hueco_L1_a0pX
concreto_fc150_L5_a0pX
lana_L6p35_a0p3
tabique_hueco_L2_a0pX
tabique_hueco_L3_a0pX
vermiculita_LX_a0pX
concreto_fc200_L20_a0pX
aluminio
Metal Decking
fachaleta_L1_a0p7
F16 Acoustic tile


'0.8'

In [26]:
diccionario["concreto_fc150_granzon_L8_a0pX"]

{'Handle': '2e7157dc-32c2-44e3-9415-7874ececbba9',
 'Roughness': 'MediumRough',
 'Thickness m': '0.15',
 'Conductivity W/m-K': '1.3',
 'Density kg/m3': '1800.0',
 'Specific Heat J/kg-K': '1000.0',
 'Thermal Absorptance': '0.8',
 'Solar Absorptance': '0.3'}

In [ ]:
!pwd